# Key Exchange

Both symmetric encryption and HMACs both require keys to be exchanged. Alice and Bob can gain confidentiality and integrity, but only if they establish a shared secret first. This seems very inconvenient. You might guess that Alice and Bob need to meet in person and covertly exchange briefcases, like they're in a spy movie. Amazingly, this is not necessary. Alice and Bob can communicate openly and arrive at a shared secret. Even if Eve is listening to all of their communications, she'll never figure out the secret. Alice and Bob can save their secret agent disguises for Halloween.

Methods to do this are called **key exchange** protocols. In this notebook, we will study the Diffie-Hellman key exchange protocol.

## Diffie-Hellman

The Diffie-Hellman key exchange protocol relies on the difficulty of solving the discrete logarithm problem. As we discussed previously, we believe it is much easier to compute $g^a \bmod n$ than it is to figure out $a$ from the answer to $g^a \bmod n$.

Alice and Bob want to arrive at a secret key. They first publicly agree on two cleverly chosen parameters: a generator $g$ and a modulo $n$. If you're curious about how $g$ and $n$ are chosen, you can ask me about it. Everyone is allowed to know and use $g$ and $n$.

Let's start with small numbers to use in an example. $n=197$ and $g$=2.

In [ ]:
g=2
n=197

Alice generates a temporary secret $a$. She never tells $a$ to anyone. Next, she computes $A=g^a \bmod n$ and sends that to Bob.

In our example, let's say Alice picks $a=57$.

In [ ]:
a=57
A=(g**a)%n
print("Alice transmits:",A)

Bob similarly generates a temporary secret $b$ and sends $B=g^b \bmod n$ to Alice.

Let's say Bob picks b=100.

In [ ]:
b=100
B=(g**b)%n
print("Bob transmits:",B)

So Alice receives B and Bob receives A. Alice now computes $B^a \bmod n$. Bob computes $A^b \bmod n$.

In [ ]:
print("Alice's result:",(B**a)%n)
print("Bob's result:",(A**b)%n)

They always arrive at the same shared secret key, because $B^a=(g^b)^a=g^{ab}$ and $A^b=(g^a)^b=g^{ab}$.

## Attacking Diffie-Hellman
What does this have to do with the discrete logarithm? Consier Eve. She knows $g$ and $n$. She also knows $A$ and $B$. She does not, however know $a$ or $b$. To arrive at the shared secret key, she either needs to compute $A^b \bmod n$ or $B^a \bmod n$. Eve needs to know either $a$ or $b$, but Alice is the only person who knows $a$ and Bob is the only person who knows $b$.

Eve knows all the parts of the equation $A=g^a \mod n$ except one: $a$. She can plug in the other values and solve the equation for $a$. Unfortunately for Eve, this is the discrete log problem. She can only get $a$ if she can solve the discrete log problem. For the small numbers in our example, it's not so difficult. Eve can just guess different values for $a$.

In [ ]:
for a in range(1,n):
    if (g**a%n)==A:
        print("a is",a)

Once Eve knows a, she can compute $B^a \bmod n$ and she has the secret key.

How do we make sure Eve can't do this? We use **HUGE** numbers. For example, $n$ is usually **thousands** of bits long. How big is a number like that? See for yourself.

In [ ]:
2**2048

With numbers that big, Alice and Bob's Diffie-Hellman computations become a little slower. But not by much. Look how fast we can compute $g^a \bmod n$ for these huge numbers with Python's efficiently implemented pow function.

In [ ]:
n=0x16be26c58be832c82a5b8f06b1b7c2379823cb73dc7ac209523cff8f29a42417f8d11cdc5ce5e9ddf6d21b2368cd689b364f6aad21d42766893e70fd4e8560d6e902f4f841c3a648d03c0af95d874d639b5559dbeb58949939c7fcf24e7fd6164fbe180e37a4ae4970a9ef2a4e97d52cc80460315e29a3c5685abd282241e188882204de5e4fc0f689df2dd0619d8492b5d48c0a3e05044f483548eb9dd353c3
g=0x9adda9f0a72d1bb54bd13c9361c4fdb0b582875297d8ead208a845781bb54ceddc151af6fe3c51e4551cbee2751ac26d25ce7a07ade601a4b5c5c5c9a43f1b87
a=0x5b3c67cf96223424342c187d56242b6bbf11b9f9ebe6a2db0a20126116014803e11389e0e1bc46fd8683fad6580745d4f89fac69becc25540fc16059afdc7009
pow(g,a,n)

As you can see, big numbers won't pose much of a problem for Alice and Bob. But nobody knows how to efficiently solve the discrete log problem for large numbers. What takes Alice and Bob seconds to compute could take Eve decades to reverse.

## Mallory-In-The-Middle

We have seen that Diffie-Hellman is secure against Eve. It is not, however, secure against Mallory. An active attacker can cause a lot of trouble. To see how , let's use our example from above again. Alice and Bob generate $a=57$ and $b=100$, then compute and exchange $A$ and $B$.

In [ ]:
g=2
n=197

a=57
A=(g**a)%n

b=100
B=(g**b)%n

When Alice sends $A$, Mallory can intercept it. Similarly, when Bob sends $B$, Mallory can intercept that. In this attack, Alice and Bob never get the real values of $A$ and $B$. Instead, Mallory uses her own secret $m$ to generate $M=g^m \bmod n$. Let's say $m=122$

In [ ]:
m=122
M=(g**m)%n

Mallory sends $M$ to Alice and Bob. When Alice gets $M$, she thinks this is $B$ from Bob. Following Diffie-Hellman, she computes the shared secret like this: $B^a \bmod n$. Unfortunately, she has $M$, not $B$. What she actually computes is $M^a \bmod n$.

In [ ]:
print("Alice computes key",M**a%n)

Bob thinks he has $A$. To arrive at the secret, he should compute $A^b \bmod n$. Just like Alice, he's been duped into accepting $M$. He actually computes $M^b \bmod n$. 

In [ ]:
print("Bob computes key",M**b%n)

They no longer have a shared secret. They cannot communicate. Is that Mallory's attack? No. It's much worse than that. Mallory has $A$ and $B$. She can now compute $A^m \bmod n$ and $B^m \bmod n$.

In [ ]:
print("Mallory computes key",A**m%n)
print("Mallory computes key",B**m%n)

Alice and Bob actually share a secret keys with Mallory! Mallory can decrypt every message Alice sends to Bob and every message Bob sends to Alice. That's bad enough, but the situation is actually even worse! After decrypting messages from Alice, she can re-encrypt them and send them to Bob. Alice and Bob have no way of knowing that Mallory is intercepting their communications. She can also tamper with the the message contents, even if Alice and Bob are using HMACs. 

Mallory-in-the-Middle attacks are extremely powerful and very dangerous.

## Exercises

The exercises for this notebook are below. Don't forget to take breaks and ask questions when you need to!

1) Let's use Diffie-Hellman with parameters are $g=2$ and $p=10789$. You generate temporary secret $a=88$. I send you $B=167$. What is our shared secret?

In [ ]:
n=10789
g=2
a=88
B=167

2) Pretend you are Eve. Alice sends $A=6522$. Bob sends $B=6246$. Can you figure out $a$ and $b$? What is Alice and Bob's shared secret key?

In [ ]:
n=10789
g=2
A=6522
B=6246

3) Pretend you are Mallory. You intercept Alice sending $A=6301$ and Bob sending $B=5533$. Your temporary secret $m=1010$. What do you send to Alice? What do you send to Bob? What is your shared secret with Alice? What is your shared secret with Bob?

In [ ]:
n=10789
g=2
A=6301
B=5533
m=1010

4) Use Python's pow function to compute $g^a \bmod n$ for these very large numbers. 

In [ ]:
n=0x7f0580c7762eaec494538f1ab087b4ad59cdd0e8c66e0549808caba52571d3eb6c39ec5b007165fee1140ea6d8fc7412e1c7985bc1cca85ef52e10106d0f8bc5
g=0xbcdd7adeecaf23afd78e014dcc7a8183f9a7e905350fe057b8a126de99ebb22b
a=0x6adf689b90f1452029df97bbb9ee072d143d4ea07cb831b414865db5425a5d11